In [1]:
!pip install --upgrade pip setuptools wheel


In [2]:
!pip install fasttext-wheel



In [3]:
import fasttext
import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [4]:
df = pd.read_csv("processed.csv") 
df.head() 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '..."


In [5]:
print(type(df['tokens'].iloc[0]))


<class 'str'>


In [6]:
with open("corpus.txt", "w", encoding="utf-8") as f:
    for tokens_str in df['tokens']:
        tokens_list = ast.literal_eval(tokens_str)
        f.write(" ".join(tokens_list) + "\n")

In [7]:
model = fasttext.train_unsupervised("corpus.txt", model="skipgram")
model.save_model("fasttext_model.bin")

In [8]:
sentence = df['tokens'].iloc[0]
print(model.get_sentence_vector(sentence))

[-0.03615935  0.11604407 -0.05360122 -0.06164953  0.01958586 -0.01176732
 -0.02871853  0.02031054  0.06887828 -0.04053482 -0.04986786  0.08566239
 -0.01758471  0.03272952  0.12941726  0.03230121  0.10087018 -0.11654276
  0.06811273  0.12743552 -0.00037307  0.06389643 -0.05426058 -0.01876932
  0.00309173  0.03020708 -0.10068329  0.04728352  0.1167683  -0.01919615
 -0.0560981  -0.05396319  0.04798488 -0.06764947 -0.06602053  0.03381625
  0.01055438 -0.04878421 -0.02900987 -0.07613344  0.10282945  0.11856021
  0.04219608  0.15313147  0.20245425  0.03316878  0.07080542  0.01209688
  0.00057397  0.06974934  0.02540202 -0.04347748  0.00616614  0.00589966
 -0.08303849 -0.05371391  0.03103779 -0.03609201 -0.0153172   0.11973029
  0.09859251  0.01630195  0.05057208  0.09826413 -0.02429986  0.07360613
  0.01395211  0.07341927 -0.02277989 -0.07417087 -0.08370531  0.05217988
 -0.02904131 -0.05879862  0.02375209 -0.06228105  0.01614755 -0.01055242
  0.02052702  0.05871615 -0.04996345  0.00197972 -0

In [9]:
def get_review_vector(tokens):
    # Join tokens into a sentence (fastText expects a string, not list)
    sentence = " ".join(tokens)
    return model.get_sentence_vector(sentence)

# Apply to DataFrame
df["vector"] = df["tokens"].apply(get_review_vector)

df

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens,vector
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially...","[0.058459032, 0.028506024, 0.0549225, -0.00385..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim...","[0.044263314, 0.013135654, 0.016781278, -0.009..."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',...","[0.058937944, 0.011035831, 0.047325734, -0.006..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'...","[0.06352168, 0.01699908, 0.047772598, 0.001047..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '...","[0.052672256, 0.027406245, 0.052592043, -0.009..."
...,...,...,...,...,...,...,...,...,...,...,...,...
19657,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses,"['happy', 'snag', 'price', 'easy', 'slip', 'cu...","[0.047781914, 0.03810713, 0.03270962, 0.003892..."
19658,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits,"['reminds', 'maternity', 'clothes', 'stretchy'...","[0.056783542, 0.0158032, 0.04231972, 0.0029605..."
19659,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses,"['worked', 'glad', 'store', 'order', 'online']","[0.0795968, 0.024796747, 0.038612925, 0.004781..."
19660,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses,"['wedding', 'summer', 'medium', 'fits', 'waist...","[0.06021569, 0.0137816705, 0.045234766, -0.009..."


In [20]:
# load corpus into sentences for vectorizer
with open("corpus.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f]

# Create the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(token_pattern="[A-Za-z]+(?:[-'][A-Za-z]+)?", norm=None)
tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)

In [21]:
# Map token indices to their corresponding tokens
index_to_token = {j: t for t, j in tfidf_vectorizer.vocabulary_.items()}

# Get the embedding dimension from the model
embedding_dimension = model.get_dimension()

# Create a weight matrix for the embeddings
weight_matrix = np.zeros((tfidf_matrix.shape[0], embedding_dimension))

for i in range(tfidf_matrix.shape[0]):
    row = tfidf_matrix.getrow(i)
    if row.nnz == 0:
        continue

    vector_sum = np.zeros(embedding_dimension, dtype=np.float32)
    weight_sum = 0.0

    # Compute the weighted sum of word vectors
    for j, w in zip(row.indices, row.data):
        token = index_to_token[j]
        vec = model.get_word_vector(token)
        vector_sum += vec * float(w)
        weight_sum += float(w)

    # Normalize the vector by the weight sum
    if weight_sum > 0:
        weight_matrix[i] = vector_sum / weight_sum



In [22]:
df["weighted_embedding"] = [vec.tolist() for vec in weight_matrix]

df.head()

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name,tokens,vector,weighted_embedding
0,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses,"['high', 'hopes', 'wanted', 'work', 'initially...","[0.058459032, 0.028506024, 0.0549225, -0.00385...","[-0.11857645213603973, 0.284871906042099, -0.0..."
1,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants,"['jumpsuit', 'fun', 'flirty', 'fabulous', 'tim...","[0.044263314, 0.013135654, 0.016781278, -0.009...","[-0.15346971154212952, 0.2994864284992218, -0...."
2,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses,"['shirt', 'due', 'adjustable', 'front', 'tie',...","[0.058937944, 0.011035831, 0.047325734, -0.006...","[-0.06939209252595901, 0.23377594351768494, -0..."
3,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses,"['tracy', 'reese', 'dresses', 'petite', 'feet'...","[0.06352168, 0.01699908, 0.047772598, 0.001047...","[-0.08697155863046646, 0.23875336349010468, -0..."
4,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits,"['basket', 'hte', 'person', 'store', 'pick', '...","[0.052672256, 0.027406245, 0.052592043, -0.009...","[-0.1943133920431137, 0.248451367020607, 0.053..."
